In [2]:
!mkdir local

mkdir: cannot create directory ‘local’: File exists


In [3]:
!pip install tensorflow_decision_forests

     |████████████████████████████████| 13.4 MB 5.1 MB/s 
     |████████████████████████████████| 462 kB 81.5 MB/s 


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from wordcloud import WordCloud
from collections import Counter
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D, Flatten, Conv1D, MaxPooling1D, SimpleRNN
from keras.callbacks import Callback, EarlyStopping
import tensorflow_decision_forests as tfdf



In [5]:
data = pd.read_csv("/content/local/train.csv")

data.drop(['Genre', 'Song'],axis=1,inplace=True)
data.loc[data['Language'] == 'en']
data.drop(['Language'],axis=1,inplace=True)



In [6]:
data.Artist.value_counts()

elvis presley           1611
chris brown             1239
elvis costello           923
ella fitzgerald          874
the rolling stones       820
                        ... 
the gregory brothers       1
flamingokvintetten         1
fjeld                      1
danceplaycreate            1
crawdad republic           1
Name: Artist, Length: 11152, dtype: int64

In [7]:
data = data.groupby("Artist").filter(lambda x: (len(x) > 700 ))
len(data.Artist.value_counts())

10

In [8]:
num_words = 50000
tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~')



data["Lyrics"] = pd.concat([data['Lyrics']],axis=0).astype("str")
tokenizer.fit_on_texts(data['Lyrics'].values)

In [9]:
max_len = 500


X = tokenizer.texts_to_sequences(data['Lyrics'].values)
X = pad_sequences(X, maxlen=max_len)
Y = pd.get_dummies(data['Artist']).values


In [10]:
base_model = load_model('/content/local/model_with_embedding.h5')
base_model.optimizer.lr = 0.005
base_model.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 500, 100)          5000000   
                                                                 
 spatial_dropout1d_6 (Spatia  (None, 500, 100)         0         
 lDropout1D)                                                     
                                                                 
 lstm_12 (LSTM)              (None, 500, 64)           42240     
                                                                 
 lstm_13 (LSTM)              (None, 32)                12416     
                                                                 
 dense_43 (Dense)            (None, 10)                330       
                                                                 
Total params: 5,054,986
Trainable params: 5,054,986
Non-trainable params: 0
___________________________________________

In [11]:
base_model.fit(X, Y, epochs=6, batch_size=32, validation_split=0.2)

Epoch 1/6
232/232 [==============================] - 16s 38ms/step - loss: 1.8905 - accuracy: 0.3683 - val_loss: 1.9924 - val_accuracy: 0.2377
Epoch 2/6
232/232 [==============================] - 8s 35ms/step - loss: 1.0221 - accuracy: 0.6655 - val_loss: 1.5768 - val_accuracy: 0.4863
Epoch 3/6
232/232 [==============================] - 8s 35ms/step - loss: 0.4972 - accuracy: 0.8463 - val_loss: 1.5639 - val_accuracy: 0.5423
Epoch 4/6
232/232 [==============================] - 8s 36ms/step - loss: 0.2254 - accuracy: 0.9376 - val_loss: 1.3072 - val_accuracy: 0.6588
Epoch 5/6
232/232 [==============================] - 8s 35ms/step - loss: 0.0988 - accuracy: 0.9757 - val_loss: 1.4912 - val_accuracy: 0.6555
Epoch 6/6
232/232 [==============================] - 8s 34ms/step - loss: 0.0462 - accuracy: 0.9887 - val_loss: 1.5905 - val_accuracy: 0.6555


In [12]:
results = base_model.evaluate(X, Y, verbose=1)
print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

290/290 [==============================] - 4s 14ms/step - loss: 0.3327 - accuracy: 0.9275
    Test Loss: 0.33274
Test Accuracy: 92.75%


In [13]:
base_model_1 = load_model('/content/local/model_without_embedding.h5')
base_model_1.summary()

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_50 (Conv1D)          (None, 500, 128)          640       
                                                                 
 max_pooling1d_48 (MaxPoolin  (None, 250, 128)         0         
 g1D)                                                            
                                                                 
 conv1d_51 (Conv1D)          (None, 250, 64)           32832     
                                                                 
 max_pooling1d_49 (MaxPoolin  (None, 125, 64)          0         
 g1D)                                                            
                                                                 
 conv1d_52 (Conv1D)          (None, 125, 32)           8224      
                                                                 
 max_pooling1d_50 (MaxPoolin  (None, 62, 32)         

In [14]:
base_model_1.fit(X, Y, epochs=6, batch_size=32, validation_split=0.2)

Epoch 1/6
232/232 [==============================] - 9s 7ms/step - loss: 2.3453 - accuracy: 0.1897 - val_loss: 2.4899 - val_accuracy: 0.0841
Epoch 2/6
232/232 [==============================] - 1s 5ms/step - loss: 2.2373 - accuracy: 0.1961 - val_loss: 2.5254 - val_accuracy: 0.0841
Epoch 3/6
232/232 [==============================] - 1s 5ms/step - loss: 2.2276 - accuracy: 0.1961 - val_loss: 2.5441 - val_accuracy: 0.0841
Epoch 4/6
232/232 [==============================] - 1s 5ms/step - loss: 2.2256 - accuracy: 0.1961 - val_loss: 2.5416 - val_accuracy: 0.0841
Epoch 5/6
232/232 [==============================] - 1s 5ms/step - loss: 2.2253 - accuracy: 0.1961 - val_loss: 2.5388 - val_accuracy: 0.0841
Epoch 6/6
232/232 [==============================] - 1s 5ms/step - loss: 2.2250 - accuracy: 0.1961 - val_loss: 2.5488 - val_accuracy: 0.0841


In [15]:
results = base_model_1.evaluate(X, Y, verbose=1)
print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

290/290 [==============================] - 1s 3ms/step - loss: 2.2894 - accuracy: 0.1737
    Test Loss: 2.28941
Test Accuracy: 17.37%


New model

In [16]:
EMBEDDING_DIM = 80
model_m = Sequential()
model_m.add(Embedding(num_words, EMBEDDING_DIM, input_length=X.shape[1]))
model_m.add(SpatialDropout1D(0.2))
model_m.add(LSTM(32,  return_sequences=True))
model_m.add(LSTM(16))
model_m.add(Dense(10, activation='softmax'))
model_m.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_m.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 80)           4000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 500, 80)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 500, 32)           14464     
                                                                 
 lstm_1 (LSTM)               (None, 16)                3136      
                                                                 
 dense (Dense)               (None, 10)                170       
                                                                 
Total params: 4,017,770
Trainable params: 4,017,770
Non-trainable params: 0
______________________________________________

In [17]:
model_m.fit(X, Y, epochs=6, batch_size=32, validation_split=0.2)

Epoch 1/6
232/232 [==============================] - 11s 36ms/step - loss: 2.0553 - accuracy: 0.2843 - val_loss: 2.2533 - val_accuracy: 0.1865
Epoch 2/6
232/232 [==============================] - 7s 32ms/step - loss: 1.5859 - accuracy: 0.4536 - val_loss: 1.9326 - val_accuracy: 0.2739
Epoch 3/6
232/232 [==============================] - 7s 32ms/step - loss: 1.2015 - accuracy: 0.5878 - val_loss: 1.8752 - val_accuracy: 0.3396
Epoch 4/6
232/232 [==============================] - 7s 32ms/step - loss: 0.8486 - accuracy: 0.7073 - val_loss: 1.6978 - val_accuracy: 0.4140
Epoch 5/6
232/232 [==============================] - 7s 32ms/step - loss: 0.6980 - accuracy: 0.7588 - val_loss: 1.5996 - val_accuracy: 0.4706
Epoch 6/6
232/232 [==============================] - 7s 32ms/step - loss: 0.5265 - accuracy: 0.8208 - val_loss: 1.7019 - val_accuracy: 0.4938


In [18]:
results = model_m.evaluate(X, Y, verbose=1)
print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

290/290 [==============================] - 4s 14ms/step - loss: 0.6848 - accuracy: 0.7859
    Test Loss: 0.68481
Test Accuracy: 78.59%
